In [8]:
import numpy as np
import matplotlib.pyplot as plt

import pulp as lp
import itertools
import copy
from tqdm import tqdm, tqdm_notebook
import gurobipy as gp

In [39]:
def solve_optimization(UCB, C, L):
    U = UCB.shape[0]
    K = UCB.shape[1]
    
    m = gp.Model()
    
    x = m.addVars(U, K, vtype=gp.GRB.BINARY, name="x")
    y = m.addVars(U, K, name="y")
    
    # Define the objective function
    obj = gp.quicksum(x[u,k] * y[u,k] * UCB[u,k] for u in range(U) for k in range(K))
    m.setObjective(obj, gp.GRB.MAXIMIZE)
    
    for u in range(U):
        m.addConstr(gp.quicksum(x[u,k] for k in range(K)) <= 1)
        for k in range(K):
            m.addConstr(y[u,k] <= 1)
            m.addConstr(y[u,k] * (gp.quicksum(x[p,k] for p in range(U))) <= C[k]) # Adding small constant to avoid division by zero error
            m.addConstr(x[u,k] <= L[u,k])
        
    m.optimize()

    # Print the status of the solution
    print("Status:", m.status)

    # Print the optimal value of the objective function
    print("Optimal Value:", m.objVal)

    # Print the optimal values of the decision variables
    for v in m.getVars():
        print(v.varName, "=", v.x)